# 2003 - 2024

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
import warnings 
warnings.filterwarnings('ignore')

def extrair_salarios_gorjeta_ano(ano):
    """
    Função para extrair e limpar a tabela de salário mínimo de gorjeta para um ano específico.
    """
    url = f'https://www.dol.gov/agencies/whd/state/minimum-wage/tipped/{ano}'
    try:
        # Etapa 1: Requisição
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Etapa 2: Análise e Navegação
        soup = BeautifulSoup(response.content, 'html.parser')
        tabela = soup.table
        
        # Etapa 3: Extração para DataFrame
        if tabela:

            elementos_para_remover = tabela.find_all('td', attrs={'colspan': '6'})
    
    # Itere sobre a lista de elementos e remova cada um
            for elemento in elementos_para_remover:
                elemento.decompose()

            df = pd.read_html(str(tabela), thousands=',', decimal='.')[0]
            df.columns = [column.replace('  ',' ') for column in df.columns]
            df.columns = [column.replace('\xa0',' ') for column in df.columns]
            if 'Future Effective Date' not in df.columns:
                df['Future Effective Date'] = pd.NA
            # Etapa 4: Padronização e Limpeza
            df.dropna(how='all',axis=0,inplace=True)
            df['Ano'] = ano
            
            # Limpar colunas numéricas, removendo '$'
            # As colunas com dados numéricos são 'Basic Combined Cash Wage' e 'Tip Credit'
            return df
        else:
            print(f"Tabela não encontrada para o ano {ano}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para o ano {ano}: {e}")
        return None

# Definir o intervalo de anos para extração
anos_historicos = range(2003, 2025)
lista_dataframes = []

for ano in anos_historicos:
    print(f"Extraindo dados para o ano: {ano}...")
    df_ano = extrair_salarios_gorjeta_ano(ano)
    if df_ano is not None:
        lista_dataframes.append(df_ano)
    
    time.sleep(random.uniform(1, 3))

# Concatenar todos os DataFrames em um único
if lista_dataframes:
    df_final2 = pd.concat(lista_dataframes, ignore_index=True)
    print("Extração de dados concluída. DataFrame final criado.")
    display(df_final2.head())
else:
    print("Nenhum dado foi extraído com sucesso.")

In [ ]:
df_final

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
import warnings 
warnings.filterwarnings('ignore')

def extrair_salarios_gorjeta_ano():
    """
    Função para extrair e limpar a tabela de salário mínimo de gorjeta para um ano específico.
    """
    url = f'https://www.dol.gov/agencies/whd/state/minimum-wage/tipped'
    try:
        # Etapa 1: Requisição
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Etapa 2: Análise e Navegação
        soup = BeautifulSoup(response.content, 'html.parser')
        tabela = soup.table
        
        # Etapa 3: Extração para DataFrame
        if tabela:

            elementos_para_remover = tabela.find_all('td', attrs={'colspan': '5'})
    
    # Itere sobre a lista de elementos e remova cada um
            for elemento in elementos_para_remover:
                elemento.decompose()

            df = pd.read_html(str(tabela), thousands=',', decimal='.')[0]
            df.columns = [column.replace('  ',' ') for column in df.columns]
            df.columns = [column.replace('\xa0',' ') for column in df.columns]
            if 'Future Effective Date' not in df.columns:
                df['Future Effective Date'] = pd.NA
            # Etapa 4: Padronização e Limpeza
            df.dropna(how='all',axis=0,inplace=True)
            df['Ano'] = 2025
            
            # Limpar colunas numéricas, removendo '$'
            # As colunas com dados numéricos são 'Basic Combined Cash Wage' e 'Tip Credit'
            return df
        else:
            print(f"Tabela não encontrada para o ano {ano}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para o ano {ano}: {e}")
        return None



df_atual = extrair_salarios_gorjeta_ano()
df_atual

In [ ]:
df_tips_completo = pd.concat([df_final2, df_atual])
df_tips_completo

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extrair_tabelas_salario(url):
    """
    Extrai e limpa todas as tabelas de salário mínimo de uma URL.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontra todas as tabelas com a classe 'table-responsive dol-table'
        tabelas = soup.find_all('table', class_='minwage')
        
        if not tabelas:
            print("Nenhuma tabela encontrada com a classe 'minwage table'.")
            return None
        
        lista_dataframes = []
        
        for tabela in tabelas:
            # Remover linhas de cabeçalho extras com 'colspan'
            for td in tabela.find_all('td', attrs={'colspan': True}):
                td.decompose()
            
            # Converter a tabela para DataFrame
            df = pd.read_html(str(tabela), thousands=',', decimal='.')[0]
            # Renomear a primeira coluna para 'State'
            df = df.rename(columns={df.columns[0]: 'State'})

            # Limpeza de nomes de colunas (remoção de '\xa0')
            df.columns = df.columns.str.replace('\xa0', ' ', regex=False).str.strip()
            
            # Adicionar o DataFrame à lista
            lista_dataframes.append(df)
            
        df_final = pd.concat(lista_dataframes, ignore_index=True)
        
        # Limpeza e padronização final dos dados
        cols_anos = df_final.columns.drop('State')
        
        for col in cols_anos:
            df_final[col] = df_final[col].astype(str) \
                .str.replace('$', '', regex=False) \
                .str.replace(r'\[.+\]|\(.+\)', '', regex=True) \
                .str.split(r'[-\&/]').str[0].str.strip()
            
            df_final[col] = pd.to_numeric(df_final[col], errors='coerce')
        
        
        return df_final
    
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para a URL: {e}")
        return None

url = 'https://www.dol.gov/agencies/whd/state/minimum-wage/history'
df_completo = extrair_tabelas_salario(url)

df_longo = df_completo.melt(
    id_vars=['State'],  # Coluna a ser mantida
    var_name='Ano',     # Novo nome para as colunas que viraram linhas
    value_name='Salario Minimo' # Novo nome para os valores que viraram linhas
)

df_longo['Ano'] = df_longo['Ano'].str.extract(r'(\d{4})').astype(int)

df_final_pivotado = df_longo.pivot_table(
    index='Ano',
    columns='State',
    values='Salario Minimo'
)
df_final_pivotado.columns.name = None
df_final_pivotado = df_final_pivotado.reset_index()

print("DataFrame Final Pivotado:")
display(df_final_pivotado)
print("\nVerificando o tipo de dados e as colunas:")
print(df_final_pivotado.info())

DataFrame Final Pivotado:


,Ano,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),...,South Dakota,Texas,U.S. Virgin Islands,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,1968,2.10,18.72,1.25,1.65,1.00,1.40,1.25,1.25,1.15,...,17.00,NaN,NaN,1.00,1.40,NaN,1.60,1.00,1.25,1.20
1,1970,2.10,18.72,1.10,1.65,1.00,1.60,1.25,1.60,1.30,...,1.00,NaN,NaN,1.00,1.60,NaN,1.60,1.00,1.30,1.30
2,1972,2.10,18.72,1.20,1.65,1.00,1.85,1.60,1.60,1.60,...,1.00,1.40,NaN,1.20,1.60,NaN,1.60,1.20,1.45,1.50
3,1976,2.80,NaN,1.90,2.00,1.00,2.21,2.00,2.25,2.20,...,2.00,1.40,NaN,1.55,2.30,2.00,2.20,2.00,2.10,1.60
4,1979,3.40,NaN,2.30,2.90,1.90,2.91,2.00,2.46,2.90,...,2.30,1.40,2.90,2.20,2.90,2.35,2.30,2.20,2.80,1.60
5,1980,3.60,NaN,2.55,2.90,1.90,3.12,2.00,2.50,3.10,...,2.30,1.40,3.10,2.35,3.10,2.35,2.30,2.20,3.00,1.60
6,1981,3.85,NaN,2.70,3.35,1.90,3.37,2.00,2.50,3.35,...,2.30,1.40,3.35,2.50,3.35,2.65,2.30,2.75,3.25,1.60
7,1988,3.85,NaN,3.25,3.35,3.00,3.75,3.35,3.50,3.35,...,2.80,3.35,3.35,2.50,3.55,2.65,2.30,3.35,3.35,1.60
8,1991,4.30,NaN,3.35,4.25,3.00,4.25,3.80,3.70,3.80,...,3.80,3.35,NaN,3.80,3.85,2.65,4.25,3.35,3.80,1.60
9,1992,4.75,NaN,3.65,4.25,3.00,4.27,4.25,3.90,4.25,...,4.25,3.35,NaN,4.25,4.25,3.65,4.25,3.80,3.80,1.60



Verificando o tipo de dados e as colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 51 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Ano                   39 non-null     int64  
 1   Alaska                39 non-null     float64
 2   Arizona               21 non-null     float64
 3   Arkansas              39 non-null     float64
 4   California            39 non-null     float64
 5   Colorado              39 non-null     float64
 6   Connecticut           39 non-null     float64
 7   Delaware              39 non-null     float64
 8   District of Columbia  39 non-null     float64
 9   Federal (FLSA)        39 non-null     float64
 10  Florida               20 non-null     float64
 11  Georgia               37 non-null     float64
 12  Guam                  39 non-null     float64
 13  Hawaii                39 non-null     float64
 14  Idaho                 39 non-null

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.dol.gov/agencies/whd/minimum-wage/state"
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

states, wages = [], []

# Cada estado começa com uma tag <h2> contendo o nome
for header in soup.find_all('h2'):
    state_name = header.get_text(strip=True)
    # O salário está em um parágrafo imediatamente após, contendo "Basic Minimum Rate"
    next_p = header.find_next_sibling('p')
    if next_p and 'Basic Minimum Rate' in next_p.get_text():
        text = next_p.get_text()
        # Extrai apenas o valor, após os dois-pontos
        wage_str = text.split(':', 1)[1].strip().split()[0]
        # Remove símbolos e converte, se numérico
        wage = wage_str.replace('$', '').replace(',', '')
        try:
            wage = float(wage)
        except ValueError:
            wage = wage_str  # Mantém string caso haja texto especial (ex: "No state minimum wage law")
        states.append(state_name)
        wages.append(wage)
    next_next_p = next_p.find_next_sibling('p') if next_p else None
    if next_next_p and 'Basic Minimum Rate' in next_next_p.get_text():
        text = next_next_p.get_text()
        wage_str = text.split(':', 1)[1].strip().split()[0]
        wage = wage_str.replace('$', '').replace(',', '')
        try:
            wage = float(wage)
        except ValueError:
            wage = wage_str
        states.append(state_name)
        wages.append(wage)

df_states = pd.DataFrame({'State': states, 'Minimum Wage': wages})
#adicionar federal ultima linha
df_states = pd.concat([df_states, pd.DataFrame({'State': ['Federal (FLSA)'], 'Minimum Wage': [7.25]})], ignore_index=True)
display(df_states)


In [ ]:
ultimo_ano = df_final_pivotado['Ano'].max()
novo_ano = ultimo_ano + 1  # ex: 2025

linha_nova = pd.Series({**{"Ano": novo_ano}, **df_states.set_index("State")["Minimum Wage"].to_dict()})

# Adiciona no DataFrame pivotado
df_final_pivotado = pd.concat([df_final_pivotado, linha_nova.to_frame().T], ignore_index=True)

df_final_pivotado = df_final_pivotado.apply(
    lambda row: row.fillna(row[1:].min()), axis=1
)
df_final_pivotado.tail()

In [ ]:
df_final_pivotado.head()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

df_plot = df_final_pivotado.copy()
federal = df_plot['Federal (FLSA)']

# Vamos gerar cores diferentes para cada estado (excluindo Federal)
cores_estados = px.colors.qualitative.Dark24
estados = df_plot.columns.drop('Ano')

fig = go.Figure()

for i, estado in enumerate(estados):
    if estado == 'Federal (FLSA)':
        # Linha Federal sempre azul
        fig.add_trace(go.Scatter(
            x=df_plot['Ano'], y=df_plot[estado],
            mode='lines+markers',
            name=estado,
            line=dict(color='blue', width=3),
            marker=dict(color='blue', size=10)
        ))
    else:
        # Contorno vermelho/azul dependendo do valor vs Federal
        linha_cor = ['blue' if val > f else 'red' 
                     for val, f in zip(df_plot[estado], federal)]
        fig.add_trace(go.Scatter(
            x=df_plot['Ano'], y=df_plot[estado],
            mode='lines+markers',
            name=estado,
            line=dict(color=cores_estados[i % len(cores_estados)], width=2),
            marker=dict(
                color=cores_estados[i % len(cores_estados)],  # cor de preenchimento
                line=dict(color=linha_cor, width=3),           # contorno condicional
                size=10
            )
        ))

fig.update_layout(
    title='Evolução do Salário Mínimo (2003-2025)',
    yaxis=dict(tickformat='$,.2f'),
    xaxis_title='Ano',
    yaxis_title='Salário Mínimo (U$)',
)
fig.show()


In [ ]:
valores = df_final_pivotado.drop(columns=['Ano'])
media_salarios = round(valores.mean(numeric_only=False).sort_values(ascending=False),2)
mediana_salarios = round(valores.median(numeric_only=False).sort_values(ascending=False),2)
desvio_padrao = round(valores.std(numeric_only=False).sort_values(ascending=False),2)

print("Média dos salários mínimos por estado (2003-2025):")
display(media_salarios.head(5))
display(media_salarios.tail(5))

print("Desvio padrão dos salários mínimos por estado (2003-2025):")
display(desvio_padrao.head(5))
display(desvio_padrao.tail(5))



In [ ]:
estados_selecionados = ['Wyoming','Washington','Federal (FLSA)']
df_final = df_final_pivotado[['Ano'] + estados_selecionados]
df_final = df_final.loc[df_final['Ano'] >= 2003]
df_final

In [ ]:
melted = df_final_pivotado.melt(id_vars=['Ano'], var_name='state', value_name='minimum_wage')
melted['Ano'] = melted['Ano'].astype(int)
melted  = melted.loc[melted['Ano'] >= 2003]
melted

In [ ]:
import plotly.express as px

fig = px.line(
    df_final,
    x="Ano",
    y=df_final.columns[1:],  # todas as colunas menos "Ano"
    markers=True,
    labels={"value": "Salário Mínimo (U$)", "Ano": "Ano", "variable": "Estado"},
    title="Evolução do Salário Mínimo ao Longo dos Anos por Estado",
    width=800,
    height=500
)

fig.show()


In [ ]:
estados_americanos_nomes = us.states.STATES
df_tips_clean = df_tips_completo.copy()
df_tips_clean.drop(['Future Effective Date'], inplace=True, axis=1)
df_tips_clean.columns = ['state','combined_cash','tip_credit','minimum_wage_tip','definition','Ano']
df_tips_clean = df_tips_clean.loc[df_tips_clean['state'].str.contains('|'.join([state.name for state in estados_americanos_nomes])) | df_tips_clean['state'].str.contains('Federal', na=False)]
def clean_state_name(name):
    for state in estados_americanos_nomes:
        if state.name in name:
            return state.name
    if 'Federal' in name:
        return 'Federal (FLSA)'
    return name

def only_one_value_per_state(cel):
    return cel[:cel.find(' ')].strip() if ' ' in cel else cel.strip()

df_tips_clean['state'] = df_tips_clean['state'].apply(clean_state_name)
df_tips_clean.fillna('$0', inplace=True)
df_tips_clean['combined_cash'] = df_tips_clean['combined_cash'].str.replace('$', '', regex=False).str.strip()
df_tips_clean['combined_cash'] = df_tips_clean['combined_cash'].apply(only_one_value_per_state)
df_tips_clean['minimum_wage_tip'] = df_tips_clean['minimum_wage_tip'].str.replace('$', '', regex=False).str.strip()
df_tips_clean['minimum_wage_tip'] = df_tips_clean['minimum_wage_tip'].apply(only_one_value_per_state)


df_2025_tip = df_tips_clean[df_tips_clean['Ano'] == 2025].copy()

# Garantir que as colunas numéricas sejam do tipo float
df_2025_tip['combined_cash'] = pd.to_numeric(df_2025_tip['combined_cash'], errors='coerce')
df_2025_tip['minimum_wage_tip'] = pd.to_numeric(df_2025_tip['minimum_wage_tip'], errors='coerce')

# Criar coluna com o maior valor entre as duas
df_2025_tip['max_wage'] = df_2025_tip[['combined_cash', 'minimum_wage_tip']].max(axis=1)

# Ver resultado
df_2025_tip= df_2025_tip[['state', 'max_wage']]


In [ ]:

df_tips = df_tips_completo.loc[df_tips_completo['Jurisdiction'].str.contains('Washington|Wyomign|FEDERAL', na=False)]
df_tips.reset_index(drop=True, inplace=True)
df_tips.drop(['Future Effective Date'], inplace=True, axis=1)
df_tips.columns = ['state','combined_cash','tip_credit','minimum_wage_tip','definition','Ano']
df_tips['combined_cash'] = df_tips['combined_cash'].str.replace('$', '', regex=False).str.strip()
df_tips['combined_cash'] = pd.to_numeric(df_tips['combined_cash'], errors='coerce')
df_tips['tip_credit'] = df_tips['tip_credit'].str.replace('$', '', regex=False).str.strip()
df_tips['minimum_wage_tip'] = df_tips['minimum_wage_tip'].str.replace('$', '', regex=False).str.strip()
df_tips['minimum_wage_tip'] = pd.to_numeric(df_tips['minimum_wage_tip'], errors='coerce')
df_tips.drop(index=39, inplace=True)
df_tips['state'] = df_tips['state'].str.strip()
df_tips['state'] = df_tips['state'].str.replace(r'\s+', ' ', regex=True) 
df_tips['state'] = df_tips['state'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
df_tips.loc[df_tips['state'].str.contains('Fair Labor Standards Act', na=False), 'state'] = 'Federal (FLSA)'

df_tips['state'] = df_tips['state'].str.replace('Washington.*','Washington', regex=True)
df_tips['state'] = df_tips['state'].str.replace('Wyoming.*','Wyoming', regex=True)

df_tips.fillna(9.25, inplace=True)


df_geral = pd.merge(df_tips, melted, how='left', left_on=['state','Ano'], right_on=['state','Ano'])
df_geral = df_geral[['state','minimum_wage','combined_cash','tip_credit','minimum_wage_tip' , 'Ano']]
df_geral

In [ ]:
import pandas as pd

# Estatísticas descritivas por estado e tipo de salário
estatisticas = df_geral.groupby('state')[['minimum_wage','combined_cash']].describe()

# Estatísticas descritivas por ano
estatisticas_ano = df_geral.groupby('Ano')[['minimum_wage','combined_cash']].describe()

# Estatísticas específicas: média, mediana, desvio padrão e quartis
resumo = df_geral.groupby('state').agg(
    media_minimum_wage=('minimum_wage','mean'),
    mediana_minimum_wage=('minimum_wage','median'),
    std_minimum_wage=('minimum_wage','std'),
    q1_minimum_wage=('minimum_wage', lambda x: x.quantile(0.25)),
    q3_minimum_wage=('minimum_wage', lambda x: x.quantile(0.75)),
    
    media_combined_cash=('combined_cash','mean'),
    mediana_combined_cash=('combined_cash','median'),
    std_combined_cash=('combined_cash','std'),
    q1_combined_cash=('combined_cash', lambda x: x.quantile(0.25)),
    q3_combined_cash=('combined_cash', lambda x: x.quantile(0.75)),
).round(2)

display(resumo)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
estados_selecionados = ['Federal (FLSA)', 'Washington', 'Wyoming']
df_filtrado = df_geral[df_geral['state'].isin(estados_selecionados)]

fig = px.line(df_filtrado, x='Ano', y='minimum_wage', color='state', markers=True,
              labels={'minimum_wage':'Salário Mínimo (U$)', 'Ano':'Ano', 'state':'Estado'},
              title='Evolução do Salário Mínimo x Gorjeta (Estados Selecionados)')

# Adiciona linhas de gorjeta
for estado in estados_selecionados:
    df_temp = df_filtrado[df_filtrado['state']==estado]
    fig.add_trace(
        go.Scatter(
            x=df_temp['Ano'],
            y=df_temp['combined_cash'].round(2),  # arredondar os valores
            mode='lines+markers',
            name=f"{estado} (Tip)",
            line=dict(dash='dot'), 
            )
    )

fig.show()


In [ ]:
import plotly.express as px
import pandas as pd
import us  # pip install us

# Supondo que df_states já exista e tenha a coluna 'State'

# Função para converter nome de estado em abreviação
def get_state_code(name):
    state = us.states.lookup(name)
    return state.abbr if state else None

# Criar coluna de código automaticamente
df_states['state_code'] = df_states['state'].apply(get_state_code)

# Criar mapa coroplético
fig = px.choropleth(
    df_states,
    locations='state_code',
    locationmode="USA-states",
    color='Minimum Wage',
    color_continuous_scale='Blues',
    scope="usa",
    labels={'Minimum Wage':'Salário Mínimo (U$)'},
    title='Salário Mínimo por Estado',
    width=800,
    height=500
)

fig.show()


In [ ]:
df_2025_tip

import plotly.express as px
import pandas as pd
import us  # pip install us

# Supondo que df_states já exista e tenha a coluna 'State'

# Função para converter nome de estado em abreviação
def get_state_code(name):
    state = us.states.lookup(name)
    return state.abbr if state else None

# Criar coluna de código automaticamente
df_2025_tip['state_code'] = df_2025_tip['state'].apply(get_state_code)

# Criar mapa coroplético

fig = px.choropleth(
    df_2025_tip,
    locations='state_code',
    locationmode="USA-states",
    color='max_wage',
    color_continuous_scale='Blues',
    scope="usa",
    labels={'max_wage':'Salário Mínimo - Gorjeta (U$)'},
    title='Salário Mínimo (Gorjeta) por Estado',
    width=800,
    height=500
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Supondo que df_final_pivotado seja seu DataFrame
df_2025 = df_final_pivotado[df_final_pivotado['Ano'] <= 2025]
estados = [col for col in df_2025.columns if col not in ['Ano', 'Federal (FLSA)']]

# Criar DataFrame para o heatmap
heatmap_df = df_2025.copy()
for estado in estados:
    heatmap_df[estado] = heatmap_df.apply(
        lambda row: 1 if row[estado] > row['Federal (FLSA)'] else 0,
        axis=1
    )

# Transformar em formato longo
heatmap_long = heatmap_df.melt(id_vars='Ano', value_vars=estados, var_name='Estado', value_name='Ultrapassou')

# Mapear valores para categorias
heatmap_long['Status'] = heatmap_long['Ultrapassou'].map({1: 'Ultrapassou', 0: 'Mantém Federal'})

# Criar heatmap
fig = px.imshow(
    heatmap_df[estados].T,  # Transpõe para ter estados nas linhas
    labels=dict(x="Ano", y="Estado", color="Status"),
    x=heatmap_df['Ano'],
    y=estados,
    color_continuous_scale=[[0, 'lightblue'], [1, 'orange']],
    aspect="auto",
)

fig.update_layout(
    title="Histórico do Salário Mínimo por Estado vs Federal (2003-2025)",
    xaxis_title="Ano",
    yaxis_title="Estado",
    height=800,
    width=1000,
    coloraxis_colorbar=dict(
        tickvals=[0, 1],
        ticktext=['Mantém Federal', 'Ultrapassou']
    )
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Preparar os dados como antes
df_2025 = df_final_pivotado[df_final_pivotado['Ano'] <= 2025]
estados = [col for col in df_2025.columns if col not in ['Ano', 'Federal (FLSA)']]

heatmap_df = df_2025.copy()
for estado in estados:
    heatmap_df[estado] = heatmap_df.apply(
        lambda row: 1 if row[estado] > row['Federal (FLSA)'] else 0,
        axis=1
    )

z = heatmap_df[estados].T.values

# Criar heatmap
fig = px.imshow(
    z,
    labels=dict(x="Ano", y="Estado", color="Status"),
    x=heatmap_df['Ano'],
    y=estados,
    color_continuous_scale=['lightblue', 'orange'],
    aspect="auto"
)

# Legenda discreta
fig.update_coloraxes(
    colorbar_tickvals=[0, 1],
    colorbar_ticktext=['Mantém Federal', 'Ultrapassou']
)

# Adicionar linhas verticais a cada década
anos = heatmap_df['Ano'].tolist()
decadas = [ano for ano in anos if ano % 3 == 0]
for dec in decadas:
    fig.add_vline(
        x=dec,
        line_width=1,
        line_dash="dash",
        line_color="grey",
        opacity=0.5
    )

# Ajustar ticks do eixo x (anos)
fig.update_xaxes(
    tickvals=decadas,
    ticktext=[str(int(ano)) for ano in decadas]
)

fig.update_layout(
    title="Histórico do Salário Mínimo por Estado vs Federal (1968-2025)",
    xaxis_title="Ano",
    yaxis_title="Estado",
    height=800,
    width=1000
)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="lightgrey")
# Adicionar linhas horizontais entre estados
for i in range(len(estados)):
    fig.add_hline(
        y=i-0.5,  # posição entre as linhas
        line_width=1,
        line_color="grey",
        opacity=0.3
    )


fig.show()


In [ ]:
# Conta quantos 1s e 0s tem em cada ano (linha)
contagem = heatmap_df.set_index("Ano").apply(pd.Series.value_counts, axis=1).fillna(0)
contagem = contagem.astype(int)

# Máximos
max_1 = contagem[1].max()
max_0 = contagem[0].max()

# Todos os anos que atingiram o máximo
anos_mais_1 = contagem[contagem[1] == max_1].index.tolist()
anos_mais_0 = contagem[contagem[0] == max_0].index.tolist()

print(f"Anos com mais 1s ({max_1} ocorrências): {anos_mais_1}")
print(f"Anos com mais 0s ({max_0} ocorrências): {anos_mais_0}")


In [ ]:
heatmap_df.loc[heatmap_df['Ano'] == 2025].T.value_counts()

In [ ]:
df_tips_clean['combined_cash'] = pd.to_numeric(df_tips_clean['combined_cash'], errors='coerce')
df_tips_clean['minimum_wage_tip'] = pd.to_numeric(df_tips_clean['minimum_wage_tip'], errors='coerce')

# Criar coluna com o maior valor entre as duas
df_tips_clean['max_wage'] = df_tips_clean[['combined_cash', 'minimum_wage_tip']].max(axis=1)

df_tips_clean= df_tips_clean[['state', 'max_wage','Ano']]

In [ ]:
import plotly.express as px

# Filtrar apenas 2025 (já feito no seu código)
todos = pd.merge(df_tips_clean, melted, how='left', left_on=['state','Ano'], right_on=['state','Ano'])
todos = todos.loc[todos['Ano'] == 2025]

# Calcular correlação
corr = todos[['max_wage', 'minimum_wage']].corr().iloc[0,1]
print(f"Correlação entre salário mínimo para gorjeta e salário mínimo regular em 2025: {corr:.2f}")

# Scatter em Plotly
fig = px.scatter(
    todos,
    x='max_wage',
    y='minimum_wage',
    color='state',
    hover_name='state',
    title=f"Relação entre Salário Mínimo para Gorjeta e Regular (2025)",
    labels={
        "max_wage": "Salário Mínimo para Gorjeta (U$)",
        "minimum_wage": "Salário Mínimo Regular (U$)"
    },
    size_max=12,
    width=800,
    height=600
)

# Melhorar visual
fig.update_traces(marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title="Estado",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Juntar bases
todos = pd.merge(
    df_tips_clean,
    melted,
    how='left',
    left_on=['state','Ano'],
    right_on=['state','Ano']
)

# --- Correlação em 2025 ---
todos_2025 = todos.loc[todos['Ano'] == 2025]
corr_2025 = todos_2025[['max_wage','minimum_wage']].corr().iloc[0,1]

# --- Correlação histórica (2003–2025) ---
corr_hist = todos[['max_wage','minimum_wage']].corr().iloc[0,1]

print(f"Correlação em 2025: {corr_2025:.2f}")
print(f"Correlação histórica (2003–2025): {corr_hist:.2f}")

# --- Gráfico estático (todos os anos juntos) ---
fig = px.scatter(
    todos,
    x='max_wage',
    y='minimum_wage',
    color='state',
    hover_name='state',
    title=f"Relação entre Salário Mínimo para Gorjeta e Regular (2003–2025)\n"
          f"Correlação Histórica: {corr_hist:.2f} | Correlação em 2025: {corr_2025:.2f}",
    labels={
        "max_wage": "Salário Mínimo para Gorjeta (U$)",
        "minimum_wage": "Salário Mínimo Regular (U$)"
    },
    width=850,
    height=600
)

# Melhorar visual
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title="Estado",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Salvar em PNG (ou PDF para melhor qualidade)
fig.write_image("salario_gorjeta_vs_regular.png", scale=2)   # PNG
# fig.write_image("salario_gorjeta_vs_regular.pdf")          # PDF (vetorial)


In [ ]:
import plotly.express as px
import pandas as pd
import us  # pip install us

# Supondo que df_states já exista e tenha a coluna 'State'

# Função para converter nome de estado em abreviação
def get_state_code(name):
    state = us.states.lookup(name)
    return state.abbr if state else None

# Criar coluna de código automaticamente
df_states['state_code'] = df_states['state'].apply(get_state_code)

# Criar mapa coroplético
fig = px.choropleth(
    df_states,
    locations='state_code',
    locationmode="USA-states",
    color='Minimum Wage',
    color_continuous_scale='Blues',
    scope="usa",
    labels={'Minimum Wage':'Salário Mínimo (U$)'},
    title='Salário Mínimo por Estado'
)

fig.show()


In [ ]:
media_salarios_minimos_gorjeta = round(df_tips['combined_cash'].mean(),2)
media_salarios_minimos_gorjeta

In [ ]:
#gráficos de disperção
import plotly.express as px
import plotly.graph_objects as go
def hex_to_rgba(hex_color, alpha=0.5):
    hex_color = hex_color.lstrip('#')
    r, g, b = int(hex_color[:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)
    return f'rgba({r},{g},{b},{alpha})'
# Paleta base para os estados (você pode escolher qualquer esquema de cores)
cores_estados = {
    'District of Columbia': '#1f77b4',
    'Federal (FLSA)': '#ff7f0e',
    'Wyoming': '#2ca02c',
    # adicione os outros estados
}
# Paleta para as linhas de gorjeta (ligeiramente mais claras ou mais suaves)
cores_tips = {estado: hex_to_rgba(cor, alpha=0.6
) for estado, cor in cores_estados.items()}

print(cores_tips)

# Gráfico do salário mínimo
fig = px.scatter(
    df_geral,
    x="Ano",
    y="minimum_wage",
    color='state',
    labels={"minimum_wage": "Salário Mínimo (U$)", "Ano": "Ano", "state": "Estado"},
    title="Evolução do Salário Mínimo x Gorjeta ao Longo dos Anos - Dispersão",
    color_discrete_map=cores_estados
)
# Adiciona linhas de gorjeta
for estado in df_geral['state'].unique():
    df_temp = df_geral[df_geral['state'] == estado]
    fig.add_trace(
        go.Scatter(
            x=df_temp['Ano'],
            y=df_temp['combined_cash'],
            mode='markers',
            name=f"{estado} (Tip)",
            marker=dict(color=cores_tips[estado], symbol='circle', size=10)
        )
    )
fig.show()